# Deep Fake
This is a dataset that has real and computer generated images and the task is to be able to differenciate between these images

## Normal CNN
The normal CNN model failed to differenciate between the CNN faile to differenctiate between them


In [1]:
import torch 
import torch.nn as nn
import torch.nn.functional as F 
from torchsummary import summary
from torch.utils.data import DataLoader

import cv2 
import numpy as np
import os
import pandas as pd 
import random

from tqdm.auto import trange, tqdm

In [2]:
class ResConvBlock(nn.Module):
    def __init__(self, filter, kernel=3):
        super(ResConvBlock, self).__init__()
        self.conv1 = nn.Conv2d(filter, filter, kernel)
        self.conv2 = nn.Conv2d(filter, filter, kernel)
        self.pad = nn.ZeroPad2d(1)
        self.pool = nn.MaxPool2d(2,2)
        self.norm = nn.BatchNorm2d(filter)
        self.lrelu = nn.LeakyReLU(0.1)

    def forward(self, x):
        s = x
        x = self.lrelu(self.norm(self.pad(self.conv1(x))))
        x = self.lrelu(self.norm(self.pad(self.conv2(x)))+s)
        return x

In [3]:
class ResBottleNeck(nn.Module):
    def __init__(self, infilter, outfilter, kernel=3):
        super(ResBottleNeck, self).__init__()
        self.conv1 = nn.Conv2d(infilter, outfilter, kernel)
        self.conv2 = nn.Conv2d(outfilter, outfilter, kernel)
        self.conv = nn.Conv2d(infilter, outfilter, 1)
        self.pad = nn.ZeroPad2d(1)
        self.norm = nn.BatchNorm2d(outfilter)
        self.lrelu = nn.LeakyReLU(0.1)
        self.pool = nn.MaxPool2d(2,2)

    def forward(self, x):
        s = x
        x = self.pool(self.lrelu(self.norm(self.pad(self.conv1(x)))))
        s = self.pool(self.lrelu(self.norm(self.conv(s))))
        x = self.lrelu(self.norm(self.pad(self.conv2(x)))+s)
        return x

In [4]:
class ResBlock(nn.Module):
    def __init__(self, infilter, outfilter, kernel=3):
        super(ResBlock, self).__init__()
        self.conv0 = ResConvBlock(infilter)
        self.conv1 = ResConvBlock(infilter)
        self.conv2 = ResConvBlock(infilter)
        self.conv3 = ResConvBlock(infilter)
        self.conv4 = ResConvBlock(infilter)
        self.conv = ResBottleNeck(infilter, outfilter)

    def forward(self, x):
        return self.conv(self.conv4(self.conv3(self.conv2(self.conv1(self.conv0(x))))))

In [30]:
class Fake(nn.Module):
    def __init__(self, filter, ndim):
        super(Fake, self).__init__()
        self.conv = nn.Conv2d(3, filter, 7)
        self.pad = nn.ZeroPad2d(3)
        self.lrelu = nn.LeakyReLU(0.1)
        self.norm = nn.BatchNorm2d(filter)
        self.pool = nn.MaxPool2d(2,2)
        self.conv1 = ResBlock(filter, filter*2, 3)
        self.conv2 = ResBlock(filter*2, filter*4, 3)
        self.conv3 = ResBlock(filter*4, filter*8, 3)
        self.conv4 = ResBlock(filter*8, filter*16, 3)
        self.avgpool = nn.AvgPool2d(4, 3)
        self.flat = nn.Flatten()
        self.dense = nn.Linear(4096, ndim)
        self.relu = nn.ReLU()
        self.final = nn.Linear(ndim, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.pool(self.lrelu(self.norm(self.pad(self.conv(x)))))
        x = self.flat(self.avgpool(self.conv4(self.conv3(self.conv2(self.conv1(x))))))
        x = self.sigmoid(self.final(self.relu(self.dense(x))))
        return x

In [31]:
model = Fake(64, 200)
summary(model, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 218, 218]           9,472
         ZeroPad2d-2         [-1, 64, 224, 224]               0
       BatchNorm2d-3         [-1, 64, 224, 224]             128
         LeakyReLU-4         [-1, 64, 224, 224]               0
         MaxPool2d-5         [-1, 64, 112, 112]               0
            Conv2d-6         [-1, 64, 110, 110]          36,928
         ZeroPad2d-7         [-1, 64, 112, 112]               0
       BatchNorm2d-8         [-1, 64, 112, 112]             128
         LeakyReLU-9         [-1, 64, 112, 112]               0
           Conv2d-10         [-1, 64, 110, 110]          36,928
        ZeroPad2d-11         [-1, 64, 112, 112]               0
      BatchNorm2d-12         [-1, 64, 112, 112]             128
        LeakyReLU-13         [-1, 64, 112, 112]               0
     ResConvBlock-14         [-1, 64, 1

In [2]:
CSV = r"C:\Users\suyash\Desktop\deep fake\metadata.csv"
dataset = pd.read_csv(CSV)
dataset.head()
FOLDER = r"C:\Users\suyash\Desktop\deep fake\deepfake"
files = FOLDER+ "\\" + dataset["videoname"].values
dataset.loc[dataset["label"]=="FAKE", "label"] = 0
dataset.loc[dataset["label"]=="REAL", "label"] = 1
vals = dataset["label"].values

In [3]:
xtrain = []
ytrain = []
for c, (file, v) in enumerate(zip(files, vals)):
    if v == 1 or c%4 == 0:
        ytrain.append(v)
        file = file[:-3]+"jpg"
        img = cv2.resize(cv2.imread(file), (224,224))/255.0
        xtrain.append(img)
        if len(ytrain) >= 5000:
            break

# converting to rch satatement removed         
X = np.array(xtrain, dtype="float32")
Y = np.array(ytrain, dtype="float32")
np.save(r"C:\Users\suyash\Desktop\X.npy", X)
np.save(r"C:\Users\suyash\Desktop\Y.npy", Y)

In [4]:
X

array([[[[0.43529412, 0.44313726, 0.44313726],
         [0.43529412, 0.44313726, 0.44313726],
         [0.4392157 , 0.44705883, 0.44705883],
         ...,
         [0.03921569, 0.03137255, 0.05490196],
         [0.04313726, 0.03529412, 0.05882353],
         [0.04705882, 0.03921569, 0.0627451 ]],

        [[0.43529412, 0.44313726, 0.44313726],
         [0.43529412, 0.44313726, 0.44313726],
         [0.4392157 , 0.44705883, 0.44705883],
         ...,
         [0.03921569, 0.03137255, 0.05490196],
         [0.04313726, 0.03529412, 0.05882353],
         [0.04705882, 0.03921569, 0.0627451 ]],

        [[0.43529412, 0.44313726, 0.44313726],
         [0.43529412, 0.44313726, 0.44313726],
         [0.4392157 , 0.44705883, 0.44705883],
         ...,
         [0.03921569, 0.03137255, 0.05490196],
         [0.04313726, 0.03529412, 0.05882353],
         [0.04705882, 0.03921569, 0.0627451 ]],

        ...,

        [[0.05490196, 0.01176471, 0.09019608],
         [0.05490196, 0.01176471, 0.09019608]

In [29]:
c=0
for y in Y[:4500]:
    if y==0:
        c=c+1
print(c)

2494


In [32]:
epochs = 3
learning_rate = 1e-3
decay = 1
criterian = nn.BCELoss()
batchSize = 10

In [ ]:
xtrain = DataLoader(X[:4500],batch_size = batchSize)
ytrain = DataLoader(Y[:4500], batch_size=batchSize)

xtest = DataLoader(X[4500:],batch_size = batchSize)
ytest = DataLoader(Y[4500:], batch_size=batchSize)

xtrain = np.array([xtr for xtr in xtrain])
ytrain = np.array([ytr for ytr in ytrain])
xtest = np.array([xtr for xtr in xtest])
ytest = np.array([ytr for ytr in ytest])

In [33]:
steps = len(xtrain)
vsteps = len(xtest)
for epoch in range(epochs):
    lss = 0
    vls = 0
    learning_rate = learning_rate/(epoch*decay+1)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    print(f"Epoch {epoch+1}/{epochs}:")
    train = trange(steps, desc=f"\tTraining: 0/{steps} steps || Loss: NaNaN || Progress", unit="step", ncols=1000)
    for c in train:
        xtr = xtrain[c]
        ytr = ytrain[c]
        ypred = model(xtr.reshape(batchSize, 3, 224, 224))
        loss = criterian(ypred, ytr.reshape(-1,1))
        lss = lss + loss    
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train.set_description(f"\tTraining: {c+1}/{steps} steps || Loss: {lss/(c+1):.4f} || Progress", refresh=True)

    test = trange(vsteps, desc=f"\tValidation: 0/{vsteps} steps || Loss: NaNaN || Progress:", unit="step", ncols=1000)
    with torch.no_grad():
        for c in test:
            xtr = xtest[c]
            ytr = ytest[c]        
            ypred = model(xtr.reshape(batchSize, 3, 224, 224))
            loss = criterian(ypred, ytr.reshape(-1,1))
            vls = vls + loss 
            test.set_description(f"\tValidation: {c+1}/{vsteps} steps || Loss: {vls/(c+1):.4f} || Progress", refresh=True)

    print("\n\n")


Epoch 1/3:


	Training: 0/450 steps || Loss: NaNaN || Progress:   0%|                                                      …

KeyboardInterrupt: 

In [34]:
for c in range(100, 200):
    print(model(X[c].reshape(-1,3,224,224)), Y[c])

tensor([[0.3002]], grad_fn=<SigmoidBackward0>) tensor(0.)
tensor([[0.5245]], grad_fn=<SigmoidBackward0>) tensor(0.)
tensor([[0.2799]], grad_fn=<SigmoidBackward0>) tensor(0.)
tensor([[0.4702]], grad_fn=<SigmoidBackward0>) tensor(1.)
tensor([[0.2703]], grad_fn=<SigmoidBackward0>) tensor(1.)
tensor([[0.3770]], grad_fn=<SigmoidBackward0>) tensor(0.)
tensor([[0.3524]], grad_fn=<SigmoidBackward0>) tensor(0.)
tensor([[0.2999]], grad_fn=<SigmoidBackward0>) tensor(1.)
tensor([[0.3117]], grad_fn=<SigmoidBackward0>) tensor(0.)
tensor([[0.3554]], grad_fn=<SigmoidBackward0>) tensor(1.)
tensor([[0.3708]], grad_fn=<SigmoidBackward0>) tensor(0.)
tensor([[0.3646]], grad_fn=<SigmoidBackward0>) tensor(0.)
tensor([[0.3198]], grad_fn=<SigmoidBackward0>) tensor(1.)
tensor([[0.3661]], grad_fn=<SigmoidBackward0>) tensor(1.)
tensor([[0.3727]], grad_fn=<SigmoidBackward0>) tensor(0.)
tensor([[0.3319]], grad_fn=<SigmoidBackward0>) tensor(1.)
tensor([[0.3698]], grad_fn=<SigmoidBackward0>) tensor(0.)
tensor([[0.283